In [1]:
import tensorflow as tf
import numpy as np

In [2]:
print(tf.__version__)

2.8.0


In [3]:
adapt_data = np.array([1., 2., 3., 4., 5.], dtype='float32')

In [4]:
input_data = np.array([1., 2., 3.], dtype='float32')

In [5]:
layer = tf.keras.layers.Normalization(axis=None)

In [6]:
layer.adapt(adapt_data)

In [7]:
layer(input_data)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.4142135 , -0.70710677,  0.        ], dtype=float32)>

In [8]:
t1 = [[0, 0, 0], [1, 1, 1]]
t2 = [[2, 2, 2], [3, 3, 3]]

In [9]:
t1 = tf.cast(t1, dtype=tf.float32)
t1

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [1., 1., 1.]], dtype=float32)>

In [10]:
t2 = tf.cast(t2, dtype=tf.float32) 
t2

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [3., 3., 3.]], dtype=float32)>

In [11]:
values=[]
values.append(t1)
values.append(t2)
values

[<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0., 0., 0.],
        [1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[2., 2., 2.],
        [3., 3., 3.]], dtype=float32)>]

In [21]:
fun = tf.stack(values, axis=1)
fun

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [2., 2., 2.]],

       [[1., 1., 1.],
        [3., 3., 3.]]], dtype=float32)>